# Hotel Recommendation System

Based on costumer reviews


Original post
https://thecleverprogrammer.com/2021/02/13/hotel-recommendation-system-with-machine-learning/#google_vignette

In [58]:
#Import libraries

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from ast import literal_eval

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\isa_m\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\isa_m\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\isa_m\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [2]:
#Set this option to see the whole content of the Hotel_Address column

pd.set_option('display.max_colwidth', 100)

In [40]:
#Read Dataset

df = pd.read_csv('Hotel_Reviews.csv')
df

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available via all possible sites i use when planing my trip...,397,1403,Only the park outside of the hotel was beautiful,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double Room ', ' Stayed 6 nights ']",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great location surroundings rooms amenities and service ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double Room ', ' Stayed 4 nights ']",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficult as most rooms are two story with narrow steps So...,42,1403,Location was good and staff were ok It is cute hotel the breakfast range is nice Will go back,21,9,7.1,"[' Leisure trip ', ' Family with young children ', ' Duplex Double Room ', ' Stayed 3 nights ', ...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk barefoot on the floor which looked as if it was not ...,210,1403,Great location in nice surroundings the bar and restaurant are nice and have a lovely outdoor a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex Double Room ', ' Stayed 3 nights ']",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line you showed me pictures of a room I thought I was ge...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' Stayed 2 nights ', ' Submitted from a mobile device ']",10 days,52.360576,4.915968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515733,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 Vienna Austria,168,8/30/2015,8.1,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the luggage to the room,14,2823,location,2,8,7.0,"[' Leisure trip ', ' Family with older children ', ' 2 rooms ', ' Stayed 5 nights ']",704 day,48.203745,16.335677
515734,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 Vienna Austria,168,8/22/2015,8.1,Atlantis Hotel Vienna,Estonia,The hotel looks like 3 but surely not 4,11,2823,Breakfast was ok and we got earlier check in,11,12,5.8,"[' Leisure trip ', ' Family with young children ', ' Standard Triple Room ', ' Stayed 2 nights ']",712 day,48.203745,16.335677
515735,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 Vienna Austria,168,8/19/2015,8.1,Atlantis Hotel Vienna,Egypt,The ac was useless It was a hot week in vienna and it only gave more hot air,19,2823,No Positive,0,3,2.5,"[' Leisure trip ', ' Family with older children ', ' 2 rooms ', ' Stayed 4 nights ', ' Submitted...",715 day,48.203745,16.335677
515736,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 Vienna Austria,168,8/17/2015,8.1,Atlantis Hotel Vienna,Mexico,No Negative,0,2823,The rooms are enormous and really comfortable I believe that a family of 5 members could be mor...,25,3,8.8,"[' Leisure trip ', ' Group ', ' Standard Triple Room ', ' Stayed 3 nights ']",717 day,48.203745,16.335677


In [41]:
#Replacing United Kingdom by UK
df.Hotel_Address = df.Hotel_Address.str.replace('United Kingdom', 'UK')

#Create new column 'Country'
df['Country'] = df['Hotel_Address'].apply(lambda x: x.split(' ')[-1])
df['Country'].unique()

array(['Netherlands', 'UK', 'France', 'Spain', 'Italy', 'Austria'],
      dtype=object)

In [42]:
df.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng', 'Country'],
      dtype='object')

In [43]:
#Drop not needed columns

df.drop(['Additional_Number_of_Scoring', 'Review_Date',
       'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'days_since_review', 'lat', 'lng'],axis= 1, inplace = True)

df

,Hotel_Address,Average_Score,Hotel_Name,Tags,Country
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double Room ', ' Stayed 6 nights ']",Netherlands
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double Room ', ' Stayed 4 nights ']",Netherlands
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,7.7,Hotel Arena,"[' Leisure trip ', ' Family with young children ', ' Duplex Double Room ', ' Stayed 3 nights ', ...",Netherlands
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,7.7,Hotel Arena,"[' Leisure trip ', ' Solo traveler ', ' Duplex Double Room ', ' Stayed 3 nights ']",Netherlands
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Suite ', ' Stayed 2 nights ', ' Submitted from a mobile device ']",Netherlands
...,...,...,...,...,...
515733,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 Vienna Austria,8.1,Atlantis Hotel Vienna,"[' Leisure trip ', ' Family with older children ', ' 2 rooms ', ' Stayed 5 nights ']",Austria
515734,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 Vienna Austria,8.1,Atlantis Hotel Vienna,"[' Leisure trip ', ' Family with young children ', ' Standard Triple Room ', ' Stayed 2 nights ']",Austria
515735,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 Vienna Austria,8.1,Atlantis Hotel Vienna,"[' Leisure trip ', ' Family with older children ', ' 2 rooms ', ' Stayed 4 nights ', ' Submitted...",Austria
515736,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 Vienna Austria,8.1,Atlantis Hotel Vienna,"[' Leisure trip ', ' Group ', ' Standard Triple Room ', ' Stayed 3 nights ']",Austria


In [46]:
#Create a function to convert the string of list into a normal list and then apply it to the columns 'Tags'

def impute(column):
    column = column[0]
    if (type(column) == str):
        return "".join(literal_eval(column))
    else:
        return column
    
df["Tags"] = df[["Tags"]].apply(impute, axis=1)
df.head()
    

,Hotel_Address,Average_Score,Hotel_Name,Tags,Country
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,7.7,Hotel Arena,Leisure trip Couple Duplex Double Room Stayed 6 nights,Netherlands
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,7.7,Hotel Arena,Leisure trip Couple Duplex Double Room Stayed 4 nights,Netherlands
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,7.7,Hotel Arena,Leisure trip Family with young children Duplex Double Room Stayed 3 nights Submitted from a...,Netherlands
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,7.7,Hotel Arena,Leisure trip Solo traveler Duplex Double Room Stayed 3 nights,Netherlands
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,7.7,Hotel Arena,Leisure trip Couple Suite Stayed 2 nights Submitted from a mobile device,Netherlands


In [47]:
#lowercase tags and countries for simplicity

df['Country'] = df['Country'].str.lower()
df['Tags'] = df['Tags'].str.lower()

Define function to recommend the names of hotels according to the location and the description provided by the user. 
Here the goal is not just recommend the name of the hotel but also rank it according to the user ratings


In [88]:
def recommend_hotel(location, description):
    '''both parameters are str'''
    
    #lowercase the description provided in the input
    description = description.lower() 
    
    #splits descriptions into nominally words
    word_tokenize(description) 
    
    #list of words that won´t be included in the analysis
    stop_words = stopwords.words('english')
    
    #Inicialize Lemmatizer
    lemm = WordNetLemmatizer()
    
    #Filtered is the list of words that are not Stopwords and will be included in the analysis
    filtered = {word for word in description if not word in stop_words} 
    
    #Create a set with no duplicated items from lemm
    filtered_set = set()
    for fs in filtered:
        filtered_set.add(lemm.lemmatize(fs))
        
        
    #Select country where Country == location lowercase. type(country) = dataframe
    country = df[df['Country']== location.lower()]
 
    #Set index for country
    country = country.set_index(np.arange(country.shape[0]))
    
    
    
    for i in range(country.shape[0]):
        
        #tokenize words from 'Tags'
        temp_token = word_tokenize(country["Tags"][i])
        
        #Get tokenized words
        temp_set = [word for word in temp_token if not word in stop_words]
        
        #Create empty set to receive lemmatized words from temp_set
        temp2_set = set()
        for s in temp_set:
            temp2_set.add(lemm.lemmatize(s))
            
    country.drop_duplicates(subset = 'Hotel_Name', keep = 'first', inplace = True)
    country.sort_values('Average_Score', ascending = False, inplace = True)
    country.reset_index(inplace= True)
    return country[["Hotel_Name", "Average_Score", "Hotel_Address"]].head()
                                    

In [89]:
recommend_hotel('Italy','I am going for a business trip' )

,Hotel_Name,Average_Score,Hotel_Address
0,Excelsior Hotel Gallia Luxury Collection Hotel,9.4,Piazza Duca D Aosta 9 Central Station 20124 Milan Italy
1,Room Mate Giulia,9.3,Silvio Pellico 4 Milan City Center 20121 Milan Italy
2,UNA Maison Milano,9.3,Via Mazzini 4 Milan City Center 20123 Milan Italy
3,Palazzo Parigi Hotel Grand Spa Milano,9.3,Corso Di Porta Nuova 1 Milan City Center 20121 Milan Italy
4,Hotel Spadari Al Duomo,9.3,Via Spadari 11 Milan City Center 20123 Milan Italy


In [90]:
recommend_hotel('UK','I am going on a honeymoon, I need a honeymoon suite room for 3 nights')

,Hotel_Name,Average_Score,Hotel_Address
0,Haymarket Hotel,9.6,1 Suffolk Place Westminster Borough London SW1Y 4HX UK
1,41,9.6,41 Buckingham Palace Road Westminster Borough London SW1W 0PS UK
2,Ham Yard Hotel,9.5,One Ham Yard Westminster Borough London W1D 7DT UK
3,Taj 51 Buckingham Gate Suites and Residences,9.5,Buckingham Gate Westminster Borough London SW1E 6AF UK
4,Charlotte Street Hotel,9.5,15 17 Charlotte Street Hotel Westminster Borough London W1T 1RJ UK
